In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist_base_path="/Users/zhouwencheng/Desktop/Grass/data/picture/mnist"
# 载入数据集
mnist = input_data.read_data_sets(mnist_base_path, one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
batch_size = 100  # 每批次大小
n_batch = mnist.train.num_examples // batch_size  # 每次多少批次
n_step = 21  # 总共训练多少轮

learn_rate = 1e-4  # 学习率

In [4]:
"""
x : 输入的x值
in_size : 输入大小
out_size: 输出大小 
activator_func: 激活函数
"""
def add_fc_layer(x, in_size, out_size, activator_func = None):
    W = tf.Variable(tf.truncated_normal(shape = [in_size, out_size], stddev = 0.1))
    b = tf.Variable(tf.constant(0.1, shape = [out_size]))
    mul = tf.matmul(x, W) + b
    if activator_func:
        return activator_func(mul)
    else:
        return mul

In [5]:
"""
x : 输入的x值
in_channel : 输入通道
out_channel: 输出通道
k_size: 卷积核尺寸
k_stride ：步长
activator_func: 激活函数
"""
def add_conv_layer(x,in_channel, out_channel,  k_size = 5, k_stride = 1, activator_func = None):
    W = tf.Variable(tf.truncated_normal(shape= [k_size, k_size, in_channel, out_channel], stddev = 0.1))
    b = tf.Variable(tf.constant(0.1, shape=[out_channel]))
    conv = tf.nn.conv2d(input = x, filter=W, strides=[1, k_stride, k_stride, 1], padding='SAME') + b
    if activator_func:
        return activator_func(conv)
    else:
        return conv

In [6]:
"""
x : 输入的x值 
k_size: 卷积核尺寸
k_stride ：步长 
"""
def add_pool_layer(x, k_size = 2, k_stride = 2):
    return tf.nn.max_pool(x, ksize=[1, k_size, k_size, 1], strides=[1, k_stride, k_stride, 1], padding='SAME')

In [8]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 28*28])
y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
x_flat = tf.reshape(x, shape=[-1, 28, 28, 1])

conv_1 = add_conv_layer(x=x_flat, in_channel=1, out_channel=6, k_size=5, k_stride=1, activator_func=tf.nn.relu) # 28 * 28 * 6
pool_1 = add_pool_layer(x=conv_1) # 14 * 14 * 6

x_conved_size = 14 * 14 * 6
pool_flat = tf.reshape(pool_1, shape=[-1, x_conved_size])

fc_1 = add_fc_layer(x=pool_flat, in_size=x_conved_size, out_size=512, activator_func=tf.nn.relu)

rate = tf.placeholder(dtype=tf.float32)
fc_1_drop = tf.nn.dropout(fc_1, keep_prob=rate)

# prediction = add_fc_layer(x=fc_1_drop, in_size=512, out_size=10, activator_func=tf.nn.softmax)
prediction = add_fc_layer(x=fc_1_drop, in_size=512, out_size=10, activator_func=None)



loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
train = tf.train.AdamOptimizer(learning_rate=learn_rate).minimize(loss)

correct_prediction = tf.equal(x=tf.math.argmax(y, 1), y=tf.math.argmax(prediction, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))

print("BG")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(n_step): 
        for batch in range(n_batch):
            x_train, y_train = mnist.train.next_batch(batch_size=batch_size)
            sess.run(train, feed_dict = {x:x_train, y:y_train, rate:0.5})
        x_test = mnist.test.images
        y_test = mnist.test.labels
        this_loss, acc = sess.run([loss, accuracy], feed_dict = {x:x_test, y:y_test, rate:1.0})
        print('step:', step, 'loss:', this_loss, 'accuracy:', acc)
        
    print("OK")


BG
step: 0 loss: 0.33819357 accuracy: 0.9083
step: 1 loss: 0.23892613 accuracy: 0.9308
step: 2 loss: 0.19297439 accuracy: 0.9432
step: 3 loss: 0.16542353 accuracy: 0.9503


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/1env2/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-c01be6e9a972>", line 34, in <module>
    sess.run(train, feed_dict = {x:x_train, y:y_train, rate:0.5})
  File "/Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/1env2/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 929, in run
    run_metadata_ptr)
  File "/Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/1env2/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1152, in _run
    feed_dict_tensor, options, run_metadata)
  File "/Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/1env2/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1328, in _do_run
    run_metadata)
  File "/Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/1env2/lib/python3.7/si

KeyboardInterrupt: 